In [2]:
import pandas as pd
import datetime
import random

from sqlalchemy import create_engine

from password import hoop_pwd
pwd = hoop_pwd.password

from nba_seer import *

modules imported
functions defined


In [3]:
# create sql conn and load game stats logs ----------------------

conn = create_engine('mysql+pymysql://root:%s@118.190.202.87:3306/nba_stats' % pwd)
game_stats_logs = pd.read_sql_table('game_stats_logs', conn)
game_stats_logs = game_stats_logs[game_stats_logs['GAME_TYPE'] != 'all_star']

print(str(len(game_stats_logs)) + ' player stats loaded.')

182450 player stats loaded.


In [4]:
# load player list ----------------------

all_players = nba_py.player.PlayerList(season='2017-18').info()

print('players list loaded')

players list loaded


In [5]:
def get_score(row):
    player_id = row['PERSON_ID']
    game_id = row['GAME_ID']
    stats = game_stats_logs[(game_stats_logs['PLAYER_ID'] == player_id) & 
                            (game_stats_logs['GAME_ID'] == game_id)].copy()
    #print(stats)
    stats['SCO'] = stats['PTS'] * 1 + stats['AST'] * 1.5 + \
    stats['OREB'] * 1 + stats['DREB'] * 0.7 + \
    stats['STL'] * 2 + stats['BLK'] * 1.8 + stats['TO'] * -1 + \
    stats['FGM'] * 0.4 + (stats['FGA'] - stats['FGM']) * -1 + stats['FG3M'] * 0.5
    return stats['SCO'].max()

In [6]:
game_date = game_stats_logs['GAME_DATE_EST'].drop_duplicates().apply(lambda x: x[:10])

In [8]:
players = pd.DataFrame()
for i in random.sample(list(game_date), 5):
    j = datetime.datetime.strptime(i, "%Y-%m-%d").date()
    games = get_games(j)
    players = players.append(get_players_p(games, game_stats_logs))
    print(str(j) + ' complete!')
else:
    print(len(players), ' players total!')

2013-11-23 complete!
2015-03-13 complete!
2016-01-10 complete!
2015-11-13 complete!
2015-05-01 complete!
1023  players total!


In [9]:
players['MA_20'] = players.apply(lambda x: get_ma(game_stats_logs, x, 20), axis = 1)
print('ma20 complete!')
players['MA_10'] = players.apply(lambda x: get_ma(game_stats_logs, x, 10), axis = 1)
print('ma10 complete!')
players['MA_5'] = players.apply(lambda x: get_ma(game_stats_logs, x, 5), axis = 1)
print('ma5 complete!')
players['MIN_20'] = players.apply(lambda x: get_min(game_stats_logs, x, 20), axis = 1)
print('min20 complete!')
players['MIN_10'] = players.apply(lambda x: get_min(game_stats_logs, x, 10), axis = 1)
print('min10 complete!')
players['MIN_5'] = players.apply(lambda x: get_min(game_stats_logs, x, 5), axis = 1)
print('min5 complete!')
players['MIN_COV_20'] = players.apply(lambda x: get_min_cov(game_stats_logs, x, 20), axis = 1)
print('min_cov_20 complete!')
players['SCO_COV_20'] = players.apply(lambda x: get_sco_cov(game_stats_logs, x, 20), axis = 1)
print('sco_cov_20 complete!')

players['EXP_SCO'] = round(players[['MA_20', 'MA_10', 'MA_5']].mean(axis = 1) * 
                           players[['MIN_20', 'MIN_10', 'MIN_5']].mean(axis = 1) / 36, 2)

players['ACT_SCO'] = players.apply(get_score, axis = 1)
players['SCO_DIFF'] = players['EXP_SCO'] - players['ACT_SCO']

ma20 complete!
ma10 complete!
ma5 complete!
min20 complete!
min10 complete!
min5 complete!
min_cov_20 complete!
sco_cov_20 complete!


In [11]:
players.sort_values('SCO_COV_20')

,PERSON_ID,TEAM_ID,Location,GAME_ID,Against_Team_ID,MA_20,MA_10,MA_5,MIN_20,MIN_10,MIN_5,MIN_COV_20,SCO_COV_20,EXP_SCO,ACT_SCO,SCO_DIFF
12,203912,1610612746,HOME,0021500558,1610612740,-1.18,1.09,7.75,4.09,5.05,3.97,0.661,-19.915,0.31,NaN,NaN
150,203996,1610612749,AWAY,0021500562,1610612752,-1.50,-8.91,-17.82,4.44,1.92,2.02,1.164,-8.121,-0.73,0.7,-1.43
85,203998,1610612761,HOME,0021400970,1610612748,-3.59,-13.48,0.00,3.93,1.24,1.03,1.374,-7.661,-0.33,NaN,NaN
9,203584,1610612766,HOME,0021400967,1610612741,-1.46,0.37,0.56,2.96,3.67,5.37,0.737,-6.357,-0.02,NaN,NaN
206,203467,1610612739,AWAY,0021300194,1610612759,-6.16,-6.16,-6.16,11.21,11.21,11.21,0.745,-5.405,-1.92,2.5,-4.42
36,203922,1610612755,HOME,0021400968,1610612758,-3.61,4.10,NaN,4.52,6.34,NaN,0.622,-5.265,0.04,NaN,NaN
23,203946,1610612741,AWAY,0021400967,1610612766,-12.73,0.00,NaN,2.92,1.07,NaN,1.089,-1.378,-0.35,NaN,NaN
184,201566,1610612760,HOME,0021500134,1610612755,44.22,43.29,43.01,32.53,33.89,33.52,0.230,0.168,40.26,40.8,-0.54
1,201933,1610612746,HOME,0021300187,1610612758,30.51,33.13,34.93,32.92,36.96,37.25,0.246,0.206,32.59,20.6,11.99
142,101108,1610612746,AWAY,0021400972,1610612742,36.10,38.25,38.74,36.10,35.90,35.30,0.123,0.223,37.45,27.7,9.75


In [ ]:
players['SCO_DIFF'].describe()

In [ ]:
cov_diff = pd.DataFrame()
#cov_diff['cov'] = abs(players[['MIN_COV_20', 'SCO_COV_20']].mean(axis = 1))
cov_diff['cov'] = abs(players['MIN_COV_20'] * players['SCO_COV_20'])
cov_diff['diff'] = players['SCO_DIFF']
cov_diff.sort_values('cov')

In [ ]:
%matplotlib inline
cov_diff[cov_diff['cov'] < 0.25].plot(x='cov', y='diff', kind='scatter', grid=True, figsize=(10, 7))

In [ ]:
cov_diff['cov'].corr(cov_diff['diff'])

In [ ]:
import numpy as np
import scipy.stats as stats
import pylab as pl

h = cov_diff['diff'][cov_diff['diff'].notnull()].apply(lambda x: round(x)).sort_values()

In [ ]:
h.value_counts().plot(kind='hist')